### **Proceso ETL (extracción, tranformación y limpieza de datos) del dataset [Movingto](https://www.movingto.com/digital-nomad-index)**

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

pd.options.mode.copy_on_write = True # CoW por defecto a partir de pandas 3.0.0 

In [2]:
# Extraemos y examinamos algunos de los datos que devuelve el dataset
# Aquí vemos que el dataset tiene nombres de variables que no siguen el mismo estilo de nomenclatura de los anteriores, tendremos que cambiarlos. 
# La columna de 'Taxes' tiene valores dispares, no normalizados, tendremos que pensar si los dejamos así o no. 
# La columna de Tax-free Period tiene el mismo problema. Veremos si necesitaremos convertir esos datos para el EDA. 

df_index_movingto = pd.read_csv("./data/digital-nomad-index-2024.csv")
df_index_movingto

,Rank,Country,Overall Score,Internet Speed,Cost of Living,Safety,Visa Ease,Quality of Life,Taxes,Tax-free Period
0,1,Portugal,92,90,85,95,95,95,NHR 20%,10 years
1,2,Estonia,91,95,75,90,98,92,0-20%,183 days/year
2,3,Georgia,90,85,88,82,100,80,1%,183 days/year
3,4,Spain,89,88,80,92,92,94,24%,183 days/year
4,5,Thailand,88,85,95,80,90,88,0-35%,183 days/year
5,6,Mexico,87,82,90,75,94,86,1.92-35%,183 days/year
6,7,Czech Republic,86,87,78,88,85,90,15%,183 days/year
7,8,Malaysia,85,80,92,85,90,85,0-30%,182 days/year
8,9,Croatia,84,84,76,88,88,89,24%,1 year
9,10,Costa Rica,83,79,85,80,90,88,0-25%,183 days/year


In [3]:
df_index_movingto.info() # Ninguna columna tiene nulos. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             40 non-null     int64 
 1   Country          40 non-null     object
 2   Overall Score    40 non-null     int64 
 3   Internet Speed   40 non-null     int64 
 4   Cost of Living   40 non-null     int64 
 5   Safety           40 non-null     int64 
 6   Visa Ease        40 non-null     int64 
 7   Quality of Life  40 non-null     int64 
 8   Taxes            40 non-null     object
 9   Tax-free Period  40 non-null     object
dtypes: int64(7), object(3)
memory usage: 3.3+ KB


#### **2. Renombramos la columnas del dataset con el mismo estilo que el resto de CSVs (snake case)**

In [4]:
# Creamos un diccionario con los nombres de las nuevas variables. 

rename_dict = {
    'Rank': 'rank',
    'Country': 'country',
    'Overall Score': 'overall_score',
    'Internet Speed': 'internet_speed',
    'Cost of Living': 'cost_of_living',
    'Safety': 'safety',
    'Visa Ease': 'visa_ease',
    'Quality of Life': 'quality_of_life',
    'Taxes': 'taxes',
    'Tax-free Period': 'tax_free_period'
}

# Aplicamos los nuevos nombres de variables

df_movingto = df_index_movingto.rename(columns=rename_dict)

In [5]:
df_movingto.info() # Aquí podemos ver que están todos los nombres cambiados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   rank             40 non-null     int64 
 1   country          40 non-null     object
 2   overall_score    40 non-null     int64 
 3   internet_speed   40 non-null     int64 
 4   cost_of_living   40 non-null     int64 
 5   safety           40 non-null     int64 
 6   visa_ease        40 non-null     int64 
 7   quality_of_life  40 non-null     int64 
 8   taxes            40 non-null     object
 9   tax_free_period  40 non-null     object
dtypes: int64(7), object(3)
memory usage: 3.3+ KB


#### **3. Limpieza de la columna `country`**

In [7]:
# De la revisión del CSV, podemos ver que en la columna 'country' hay un problema
# Una de las filas tiene un valor no normalizado: está formada por ciudad y país (Bali, Indonesia)
# El resto de valores de 'country' solo tienen nombres de países. 

df_movingto['country'] # índice número 12

0                 Portugal
1                  Estonia
2                  Georgia
3                    Spain
4                 Thailand
5                   Mexico
6           Czech Republic
7                 Malaysia
8                  Croatia
9               Costa Rica
10                 Germany
11             Netherlands
12         Bali, Indonesia
13                 Vietnam
14               Singapore
15                  Canada
16                   Malta
17                  Greece
18                  Cyprus
19                 Uruguay
20                  Panama
21                   Chile
22               Argentina
23                Colombia
24                   Japan
25             South Korea
26                  Taiwan
27    United Arab Emirates
28                Barbados
29              Montenegro
30                 Hungary
31                 Romania
32                Bulgaria
33                  Serbia
34               Australia
35             New Zealand
36                 Iceland
3

In [8]:
# Sustituimos el valor actual de esa fila por el correcto

df_movingto.loc[df_movingto['country'] == 'Bali, Indonesia', 'country'] = 'Indonesia'

In [9]:
df_movingto['country'] # Y comprobamos que está correcto 

0                 Portugal
1                  Estonia
2                  Georgia
3                    Spain
4                 Thailand
5                   Mexico
6           Czech Republic
7                 Malaysia
8                  Croatia
9               Costa Rica
10                 Germany
11             Netherlands
12               Indonesia
13                 Vietnam
14               Singapore
15                  Canada
16                   Malta
17                  Greece
18                  Cyprus
19                 Uruguay
20                  Panama
21                   Chile
22               Argentina
23                Colombia
24                   Japan
25             South Korea
26                  Taiwan
27    United Arab Emirates
28                Barbados
29              Montenegro
30                 Hungary
31                 Romania
32                Bulgaria
33                  Serbia
34               Australia
35             New Zealand
36                 Iceland
3

#### **4. Verificación de número de nulos en el dataset**

In [10]:
# De la revisión del dataset con .info() parece que ninguna columna tiene nulos. 
# De todas formas lo confirmamos para no cometer errores. 

null_counts = df_movingto.isnull().sum()

print("Nulos por columna:")
print(null_counts)
print(f"\nTotal nulos: {null_counts.sum()}") # El dataset está limpio de nulos. 

Nulos por columna:
rank               0
country            0
overall_score      0
internet_speed     0
cost_of_living     0
safety             0
visa_ease          0
quality_of_life    0
taxes              0
tax_free_period    0
dtype: int64

Total nulos: 0


In [11]:
# Resumen del dataset totalmente limpio 

df_movingto.head(20) 

,rank,country,overall_score,internet_speed,cost_of_living,safety,visa_ease,quality_of_life,taxes,tax_free_period
0,1,Portugal,92,90,85,95,95,95,NHR 20%,10 years
1,2,Estonia,91,95,75,90,98,92,0-20%,183 days/year
2,3,Georgia,90,85,88,82,100,80,1%,183 days/year
3,4,Spain,89,88,80,92,92,94,24%,183 days/year
4,5,Thailand,88,85,95,80,90,88,0-35%,183 days/year
5,6,Mexico,87,82,90,75,94,86,1.92-35%,183 days/year
6,7,Czech Republic,86,87,78,88,85,90,15%,183 days/year
7,8,Malaysia,85,80,92,85,90,85,0-30%,182 days/year
8,9,Croatia,84,84,76,88,88,89,24%,1 year
9,10,Costa Rica,83,79,85,80,90,88,0-25%,183 days/year


#### **5. Exportación del dataset limpio a la carpeta ./data**

In [12]:
df_movingto.to_csv("./data/digital-nomad-index-movingto-clean.csv", index=False)